# Task description
- Identity the speakers of given features.
- Main goal: Get familiar with transformer.

In [ ]:
# unzip the file
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/MyDrive/Colab Notebooks"
!tar zxvf Dataset.tar.gz

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks
Dataset/
Dataset/uttr-bf3c7ba5bd8e4b8fb22737fda57fa37f.pt


## Fix Random Seed

In [1]:
import numpy as np
import torch
import random

def set_seed(seed):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

set_seed(87)

# Data

## Dataset
- Original dataset is [Voxceleb2](https://www.robots.ox.ac.uk/~vgg/data/voxceleb/vox2.html).
- The [license](https://creativecommons.org/licenses/by/4.0/) and [complete version](https://www.robots.ox.ac.uk/~vgg/data/voxceleb/files/license.txt) of Voxceleb2.
- We randomly select 600 speakers from Voxceleb2.
- Then preprocess the raw waveforms into mel-spectrograms.

- Args:
  - data_dir: The path to the data directory.
  - metadata_path: The path to the metadata.
  - segment_len: The length of audio segment for training.
- The architecture of data directory \\
  - data directory \\
  |---- metadata.json \\
  |---- testdata.json \\
  |---- mapping.json \\
  |---- uttr-{random string}.pt \\

- The information in metadata
  - "n_mels": The dimention of mel-spectrogram.
  - "speakers": A dictionary.
    - Key: speaker ids.
    - value: "feature_path" and "mel_len"


For efficiency, we segment the mel-spectrograms into segments in the traing step.

In [ ]:
import os
import json
import torch
import random
import torch.nn as nn
from pathlib import Path
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence


class myDataset(Dataset):
	def __init__(self, data_dir, segment_len=128):
		self.data_dir = data_dir
		self.segment_len = segment_len

		# Load the mapping from speaker neme to their corresponding id.
		mapping_path = Path(data_dir) / "mapping.json"
		mapping = json.load(mapping_path.open())
		self.speaker2id = mapping["speaker2id"]

		# Load metadata of training data.
		metadata_path = Path(data_dir) / "metadata.json"
		metadata = json.load(open(metadata_path))["speakers"]

		# Get the total number of speaker.
		self.speaker_num = len(metadata.keys())
		self.data = []
		for speaker in metadata.keys():
			for utterances in metadata[speaker]:
				self.data.append([utterances["feature_path"], self.speaker2id[speaker]])

	def __len__(self):
			return len(self.data)

	def __getitem__(self, index):
		feat_path, speaker = self.data[index]
		# Load preprocessed mel-spectrogram.
		mel = torch.load(os.path.join(self.data_dir, feat_path))

		# Segmemt mel-spectrogram into "segment_len" frames.
		if len(mel) > self.segment_len:
			# Randomly get the starting point of the segment.
			start = random.randint(0, len(mel) - self.segment_len)
			# Get a segment with "segment_len" frames.
			mel = torch.FloatTensor(mel[start:start+self.segment_len])
		else:
			mel = torch.FloatTensor(mel)
		# Turn the speaker id into long for computing loss later.
		speaker = torch.FloatTensor([speaker]).long()
		return mel, speaker

	def get_speaker_number(self):
		return self.speaker_num

## Dataloader
- Split dataset into training dataset(90%) and validation dataset(10%).
- Create dataloader to iterate the data.

In [ ]:
import torch
from torch.utils.data import DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence


def collate_batch(batch):
	# Process features within a batch.
	"""Collate a batch of data."""
	mel, speaker = zip(*batch)
	# Because we train the model batch by batch, we need to pad the features in the same batch to make their lengths the same.
	mel = pad_sequence(mel, batch_first=True, padding_value=-20)    # pad log 10^(-20) which is very small value.
	# mel: (batch size, length, 40)
	return mel, torch.FloatTensor(speaker).long()


def get_dataloader(data_dir, batch_size, n_workers):
	"""Generate dataloader"""
	dataset = myDataset(data_dir)
	speaker_num = dataset.get_speaker_number()
	# Split dataset into training dataset and validation dataset
	trainlen = int(0.9 * len(dataset))
	lengths = [trainlen, len(dataset) - trainlen]
	trainset, validset = random_split(dataset, lengths)

	train_loader = DataLoader(
		trainset,
		batch_size=batch_size,
		shuffle=True,
		drop_last=True,
		num_workers=n_workers,
		pin_memory=True,
		collate_fn=collate_batch,
	)
	valid_loader = DataLoader(
		validset,
		batch_size=batch_size,
		num_workers=n_workers,
		drop_last=True,
		pin_memory=True,
		collate_fn=collate_batch,
	)

	return train_loader, valid_loader, speaker_num

# Model
- Base transformer encoder layer in [Attention Is All You Need](https://arxiv.org/abs/1706.03762)
    - TransformerEncoderLayer:
    - TransformerEncoder:

In [ ]:

class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, n_head):
        super(MultiHeadAttention, self).__init__()
        self.d_model = d_model
        self.n_head = n_head
        self.dim_head = d_model
        self.W_Q = nn.Linear(self.dim_head, self.dim_head * self.n_head, bias = False)
        self.W_K = nn.Linear(self.dim_head, self.dim_head * self.n_head, bias = False)
        self.W_V = nn.Linear(self.dim_head, self.dim_head * self.n_head, bias = False)
        self.fc = nn.Linear(self.n_head *self.dim_head, d_model )
    def forward(self, values, query, keys):
        N = query.shape[0]

        value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]

        # Split the embedding into self.heads different pieces
        values = values.reshape(N, value_len, self.n_head, self.dim_head)
        keys = keys.reshape(N, key_len, self.n_head, self.dim_head)
        queries = query.reshape(N, query_len, self.n_head, self.dim_head)

        values = self.W_V(values)
        keys = self.W_K(keys)
        queries = self.W_Q(queries)

        energy = torch.einsum("nqhd,nkhd->nhqk", [queries, keys])

        attention = torch.nn.functional.softmax(energy / (self.d_model ** (1 / 2)), dim=3)

        out = torch.einsum("nhql,nlhd->nqhd", [attention, values]).reshape(
            N, query_len, self.n_head * self.dim_head
        )

        out = self.fc(out)
        return out

class TransformerEncoderLayer(nn.Module):
    def __init__(self, d_model, dim_feedforward, nhead, dropout = 0.1):
        super().__init__()
        self.enc_self_attn = MultiHeadAttention(d_model , nhead)
        self.Norm1 = nn.LayerNorm(d_model)
        self.Norm2 = nn.LayerNorm(d_model)
        self.feedforward = nn.Sequential(
            nn.Linear(d_model, dim_feedforward * d_model),
            nn.ReLU(),
            nn.Linear(dim_feedforward * d_model, d_model)
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        attention = self.enc_self_attn(x,x,x)
        x = self.dropout(self.Norm1(attention + x))
        forward = self.feedforward(x)
        out = self.dropout(forward)
        return out
class TransformerEncoder(nn.Module):
    def __init__(self,d_model, encode_layer, num_layers):
        super().__init__()
        self.layers = nn.ModuleList(
            [
                encode_layer
                for _ in range(num_layers)
            ]
        )

    def forward(self, x):
        N = x.shape
        for layer in self.layers:
            x = layer(x)
        return x

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Classifier(nn.Module):
	def __init__(self, d_model= 80, n_spks=600, dropout=0.1):
		super().__init__()
		# Project the dimension of features from that of input into d_model.
		self.prenet = nn.Linear(40, d_model)
		# TODO:
		#   Build vanilla transformer encoder block from scratch !!!
        #   You can't call the transformer encoder block function from PyTorch library !!!
        #   The number of encoder layers should be less than 3 !!!
        #   The parameter size of transformer encoder block should be less than 500k !!!
		self.encoder_layer = TransformerEncoderLayer(d_model, dim_feedforward=10,nhead=1) #your own transformer encoder layer
		self.encoder = TransformerEncoder(d_model, self.encoder_layer,num_layers=3)#your own transformer encoder
		# Project the the dimension of features from d_model into speaker nums.
		self.pred_layer = nn.Sequential(
			nn.Linear(d_model, d_model),
			nn.ReLU(),
			nn.Linear(d_model, n_spks),
		)

	def forward(self, mels):
		out = self.prenet(mels)
		out = out.permute(1, 0, 2)
		# The encoder layer expect features in the shape of (length, batch size, d_model).
		out = self.encoder_layer(out)
		out = out.transpose(0, 1)
		# mean pooling
		stats = out.mean(dim=1)
		out = self.pred_layer(stats)
		return out

encoderblock = Classifier()
param_enc = sum(p.numel() for p in encoderblock.encoder.parameters())
print (f"The parameter size of encoder block is {param_enc/1000}k")

The parameter size of encoder block is 154.88k


# Learning rate schedule
- For transformer architecture, the design of learning rate schedule is different from that of CNN.
- Previous works show that the warmup of learning rate is useful for training models with transformer architectures.
- The warmup schedule
  - Set learning rate to 0 in the beginning.
  - The learning rate increases linearly from 0 to initial learning rate during warmup period.

In [ ]:
import math

import torch
from torch.optim import Optimizer
from torch.optim.lr_scheduler import LambdaLR


def get_cosine_schedule_with_warmup(
    optimizer: Optimizer,
    num_warmup_steps: int,
    num_training_steps: int,
    num_cycles: float = 0.5,
    last_epoch: int = -1,
):
	def lr_lambda(current_step):
		# Warmup
		if current_step < num_warmup_steps:
			return float(current_step) / float(max(1, num_warmup_steps))
		# decadence
		progress = float(current_step - num_warmup_steps) / float(
			max(1, num_training_steps - num_warmup_steps)
		)
		return max(
			0.0, 0.5 * (1.0 + math.cos(math.pi * float(num_cycles) * 2.0 * progress))
		)

	return LambdaLR(optimizer, lr_lambda, last_epoch)

# Model Function
- Model forward function.

In [ ]:
import torch


def model_fn(batch, model, criterion, device):
	"""Forward a batch through the model."""

	mels, labels = batch
	mels = mels.to(device)
	labels = labels.to(device)

	outs = model(mels)

	loss = criterion(outs, labels)

	# Get the speaker id with highest probability.
	preds = outs.argmax(1)
	# Compute accuracy.
	accuracy = torch.mean((preds == labels).float())

	return loss, accuracy

# Validate
- Calculate accuracy of the validation set.

In [ ]:
from tqdm import tqdm
import torch


def valid(dataloader, model, criterion, device):
	"""Validate on validation set."""

	model.eval()
	running_loss = 0.0
	running_accuracy = 0.0
	pbar = tqdm(total=len(dataloader.dataset), ncols=0, desc="Valid", unit=" uttr")

	for i, batch in enumerate(dataloader):
		with torch.no_grad():
			loss, accuracy = model_fn(batch, model, criterion, device)
			running_loss += loss.item()
			running_accuracy += accuracy.item()

		pbar.update(dataloader.batch_size)
		pbar.set_postfix(
			loss=f"{running_loss / (i+1):.2f}",
			accuracy=f"{running_accuracy / (i+1):.2f}",
		)

	pbar.close()
	model.train()

	return running_accuracy / len(dataloader)

# Main function

In [ ]:
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import DataLoader, random_split


def parse_args():
	"""arguments"""
	config = {
		"data_dir": "./Dataset",
		"save_path": "model.ckpt",
		"batch_size": 32,
		"n_workers": 8,
		"valid_steps": 2000,
		"warmup_steps": 1000,
		"save_steps": 10000,
		"total_steps": 70000,
	}

	return config


def main(
	data_dir,
	save_path,
	batch_size,
	n_workers,
	valid_steps,
	warmup_steps,
	total_steps,
	save_steps,
):
	"""Main function."""
	device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
	print(f"[Info]: Use {device} now!")

	train_loader, valid_loader, speaker_num = get_dataloader(data_dir, batch_size, n_workers)
	train_iterator = iter(train_loader)
	print(f"[Info]: Finish loading data!",flush = True)

	model = Classifier(n_spks=speaker_num).to(device)
	criterion = nn.CrossEntropyLoss()
	optimizer = AdamW(model.parameters(), lr=1e-3)
	scheduler = get_cosine_schedule_with_warmup(optimizer, warmup_steps, total_steps)
	print(f"[Info]: Finish creating model!",flush = True)

	best_accuracy = -1.0
	best_state_dict = None

	pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

	for step in range(total_steps):
		# Get data
		try:
			batch = next(train_iterator)
		except StopIteration:
			train_iterator = iter(train_loader)
			batch = next(train_iterator)

		loss, accuracy = model_fn(batch, model, criterion, device)
		batch_loss = loss.item()
		batch_accuracy = accuracy.item()

		# Updata model
		loss.backward()
		optimizer.step()
		scheduler.step()
		optimizer.zero_grad()

		# Log
		pbar.update()
		pbar.set_postfix(
			loss=f"{batch_loss:.2f}",
			accuracy=f"{batch_accuracy:.2f}",
			step=step + 1,
		)

		# Do validation
		if (step + 1) % valid_steps == 0:
			pbar.close()

			valid_accuracy = valid(valid_loader, model, criterion, device)

			# keep the best model
			if valid_accuracy > best_accuracy:
				best_accuracy = valid_accuracy
				best_state_dict = model.state_dict()

			pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

		# Save the best model so far.
		if (step + 1) % save_steps == 0 and best_state_dict is not None:
			torch.save(best_state_dict, save_path)
			pbar.write(f"Step {step + 1}, best model saved. (accuracy={best_accuracy:.4f})")

	pbar.close()


if __name__ == "__main__":
	main(**parse_args())

[Info]: Use cuda now!


Train:   0% 0/2000 [06:32<?, ? step/s]
Train:   0% 0/2000 [05:13<?, ? step/s]


[Info]: Finish loading data!
[Info]: Finish creating model!


Train: 100% 2000/2000 [02:43<00:00, 12.26 step/s, accuracy=0.12, loss=4.08, step=2000]
Valid: 100% 5664/5667 [00:16<00:00, 345.81 uttr/s, accuracy=0.13, loss=4.28]
Train: 100% 2000/2000 [02:41<00:00, 12.39 step/s, accuracy=0.22, loss=3.65, step=4000]
Valid: 100% 5664/5667 [00:16<00:00, 343.01 uttr/s, accuracy=0.24, loss=3.59]
Train: 100% 2000/2000 [02:41<00:00, 12.40 step/s, accuracy=0.47, loss=2.40, step=6000]
Valid: 100% 5664/5667 [00:16<00:00, 349.93 uttr/s, accuracy=0.30, loss=3.21]
Train: 100% 2000/2000 [02:42<00:00, 12.28 step/s, accuracy=0.28, loss=2.95, step=8000]
Valid: 100% 5664/5667 [00:15<00:00, 360.24 uttr/s, accuracy=0.36, loss=2.96]
Train: 100% 2000/2000 [02:39<00:00, 12.54 step/s, accuracy=0.47, loss=2.12, step=1e+4]
Valid: 100% 5664/5667 [00:17<00:00, 329.53 uttr/s, accuracy=0.39, loss=2.80]
Train:   0% 0/2000 [00:00<?, ? step/s]



Train:   0% 10/2000 [00:00<00:57, 34.36 step/s, accuracy=0.38, loss=3.13, step=1e+4]

Step 10000, best model saved. (accuracy=0.3893)


...

Step 50000, best model saved. (accuracy=0.6299)


Train: 100% 2000/2000 [02:42<00:00, 12.31 step/s, accuracy=0.66, loss=1.50, step=52000]
Valid: 100% 5664/5667 [00:16<00:00, 346.46 uttr/s, accuracy=0.64, loss=1.62]
Train: 100% 2000/2000 [02:44<00:00, 12.13 step/s, accuracy=0.50, loss=2.35, step=54000]
Valid: 100% 5664/5667 [00:16<00:00, 344.26 uttr/s, accuracy=0.64, loss=1.64]
Train: 100% 2000/2000 [02:47<00:00, 11.93 step/s, accuracy=0.66, loss=1.75, step=56000]
Valid: 100% 5664/5667 [00:16<00:00, 341.92 uttr/s, accuracy=0.63, loss=1.65]
Train: 100% 2000/2000 [02:42<00:00, 12.33 step/s, accuracy=0.72, loss=1.40, step=58000]
Valid: 100% 5664/5667 [00:16<00:00, 344.22 uttr/s, accuracy=0.66, loss=1.58]
Train: 100% 2000/2000 [02:44<00:00, 12.19 step/s, accuracy=0.56, loss=1.77, step=6e+4]
Valid: 100% 5664/5667 [00:16<00:00, 335.52 uttr/s, accuracy=0.65, loss=1.58]


Train:   0% 0/2000 [00:00<?, ? step/s]

Train:   0% 7/2000 [00:00<01:10, 28.29 step/s, accuracy=0.72, loss=0.83, step=6e+4]

Step 60000, best model saved. (accuracy=0.6555)


Train: 100% 2000/2000 [02:43<00:00, 12.26 step/s, accuracy=0.78, loss=1.04, step=62000]
Valid: 100% 5664/5667 [00:16<00:00, 344.24 uttr/s, accuracy=0.66, loss=1.55]
Train: 100% 2000/2000 [02:44<00:00, 12.16 step/s, accuracy=0.69, loss=1.04, step=64000]
Valid: 100% 5664/5667 [00:16<00:00, 338.64 uttr/s, accuracy=0.65, loss=1.59]
Train: 100% 2000/2000 [02:43<00:00, 12.20 step/s, accuracy=0.69, loss=1.04, step=66000]
Valid: 100% 5664/5667 [00:16<00:00, 340.11 uttr/s, accuracy=0.66, loss=1.56]
Train: 100% 2000/2000 [02:44<00:00, 12.19 step/s, accuracy=0.78, loss=0.87, step=68000]
Valid: 100% 5664/5667 [00:16<00:00, 335.40 uttr/s, accuracy=0.66, loss=1.54]
Train: 100% 2000/2000 [02:44<00:00, 12.16 step/s, accuracy=0.72, loss=1.48, step=7e+4]
Valid: 100% 5664/5667 [00:16<00:00, 333.63 uttr/s, accuracy=0.66, loss=1.55]


Train:   0% 0/2000 [00:00<?, ? step/s]

Train:   0% 0/2000 [00:00<?, ? step/s]


Step 70000, best model saved. (accuracy=0.6614)


# Inference

## Dataset of inference

In [ ]:
import os
import json
import torch
from pathlib import Path
from torch.utils.data import Dataset


class InferenceDataset(Dataset):
	def __init__(self, data_dir):
		testdata_path = Path(data_dir) / "testdata.json"
		metadata = json.load(testdata_path.open())
		self.data_dir = data_dir
		self.data = metadata["utterances"]

	def __len__(self):
		return len(self.data)

	def __getitem__(self, index):
		utterance = self.data[index]
		feat_path = utterance["feature_path"]
		mel = torch.load(os.path.join(self.data_dir, feat_path))

		return feat_path, mel


def inference_collate_batch(batch):
	"""Collate a batch of data."""
	feat_paths, mels = zip(*batch)

	return feat_paths, torch.stack(mels)

## Main funcrion of Inference

In [ ]:
import json
import csv
from pathlib import Path
from tqdm.notebook import tqdm

import torch
from torchsummary import summary
from torch.utils.data import DataLoader

def parse_args():
	"""arguments"""
	config = {
		"data_dir": "./Dataset",
		"model_path": "./model.ckpt",
		"output_path": "./output.csv",
	}

	return config


def main(
	data_dir,
	model_path,
	output_path,
):
	"""Main function."""
	device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
	print(f"[Info]: Use {device} now!")

	mapping_path = Path(data_dir) / "mapping.json"
	mapping = json.load(mapping_path.open())

	dataset = InferenceDataset(data_dir)
	dataloader = DataLoader(
		dataset,
		batch_size=1,
		shuffle=False,
		drop_last=False,
		num_workers=8,
		collate_fn=inference_collate_batch,
	)
	print(f"[Info]: Finish loading data!",flush = True)

	speaker_num = len(mapping["id2speaker"])
	model = Classifier(n_spks=speaker_num).to(device)
	model.load_state_dict(torch.load(model_path))
	model.eval()
	print(f"[Info]: Finish creating model!",flush = True)

	results = [["Id", "Category"]]
	for feat_paths, mels in tqdm(dataloader):
		with torch.no_grad():
			mels = mels.to(device)
			outs = model(mels)
			preds = outs.argmax(1).cpu().numpy()
			for feat_path, pred in zip(feat_paths, preds):
				results.append([feat_path, mapping["id2speaker"][str(pred)]])

	with open(output_path, 'w', newline='') as csvfile:
		writer = csv.writer(csvfile)
		writer.writerows(results)


if __name__ == "__main__":
	main(**parse_args())

[Info]: Use cuda now!
[Info]: Finish loading data!
[Info]: Finish creating model!


  0%|          | 0/8000 [00:00<?, ?it/s]